In [1]:
import fasttext

kbd_model = fasttext.load_model('../data/processed/embeddings/fasttext_skipgram_kbd_300.bin')

In [2]:
import pandas as pd

kbd_rus_glossary_df = pd.read_csv('../data/processed/glossary/html_cleaned/Ady-Ady_AP.csv')

kbd_rus_glossary_df = kbd_rus_glossary_df.rename(
    columns={
        '#name': 'word',
        'Адыгэбзэ псалъалъэ (Ady) (eastern)': 'description'}
)

In [13]:
glossary_kbd_words = set(kbd_rus_glossary_df['word'].values)
glossary_kbd_words = {word.lower().replace('i', 'I') for word in glossary_kbd_words}

In [15]:
from gensim.models import KeyedVectors
from tqdm import tqdm

# Сохраним вектора только для слов из словаря. (так мы получим похожие слова в словарной форме)
kbd_restricted_glossary_kv = KeyedVectors(vector_size=300, count=len(glossary_kbd_words))
for word in tqdm(glossary_kbd_words):
    kbd_restricted_glossary_kv.add_vector(word, kbd_model.get_word_vector(word))

kbd_restricted_glossary_kv.save('../data/processed/embeddings/fasttext_skipgram_kbd_300_restricted_glossary.kv')

100%|██████████| 27268/27268 [00:00<00:00, 89327.08it/s]


In [0]:
import json

# Сгруппируем похожие слова для каждого слова из словаря.
glossary_word_groups_by_glossary_words = {}

for word in tqdm(glossary_kbd_words):
    similar_glossary_words = [w for w, _ in kbd_restricted_glossary_kv.most_similar(word, topn=10)]
    glossary_word_groups_by_glossary_words[word] = similar_glossary_words

In [10]:
# "теплъафIэ": [
#     "теплъэншэ",
#     "теплъэ",
#     "теплъаджэ",
#     "фэтеплъэ",
#     "теплъэн",
#     "теплъызэн",
#     "фафIэ",
#     "лыфIэ",
#     "фIэрафIэ",
#     "теплъэгъуей"
# ],
with open('../data/processed/word_groups/glossary_word_groups_by_glossary_words.json', 'w') as f:
    json.dump(glossary_word_groups_by_glossary_words, f, ensure_ascii=False, indent=4)

100%|██████████| 27268/27268 [00:42<00:00, 640.93it/s]


In [17]:
# соберем слова и векторы из текста отфильтровав слова

file_name = 'freq_1000000_oshhamaho.csv'

df = pd.read_csv(f'../data/processed/word_freqs/{file_name}', sep=',')
words = df[df['freq'] > 1]['word'].values.tolist()

kbd_restricted_kv = KeyedVectors(vector_size=300, count=len(words))
for word in tqdm(words):
    kbd_restricted_kv.add_vector(word, kbd_model.get_word_vector(word))

kbd_restricted_kv.save(f'../data/processed/embeddings/fasttext_skipgram_kbd_300_restricted_{file_name}.kv')

100%|██████████| 179574/179574 [00:02<00:00, 71765.24it/s]


In [21]:
# cгруппируем похожие слова для каждого слова из текста 
# (так мы получим похожие слова и различные формы слов)

word_groups_by_glossary_words = {}

for word in tqdm(glossary_kbd_words):
    word_vector = kbd_model.get_word_vector(word)
    similar_words = [w for w, _ in kbd_restricted_kv.similar_by_vector(word_vector, topn=10)]
    word_groups_by_glossary_words[word] = similar_words

100%|██████████| 27268/27268 [05:10<00:00, 87.78it/s] 


In [22]:
# "теплъафIэ": [
#     "теплъафIэ",
#     "теплъафIэр",
#     "теплъафIэт",
#     "теплъафIэм",
#     "теплъа",
#     "теплъэф",
#     "теплъи",
#     "теплъаджэт",
#     "теплъэ…",
#     "теплъэт"
#     ],

with open('../data/processed/word_groups/word_groups_by_glossary_words.json', 'w') as f:
    json.dump(word_groups_by_glossary_words, f, ensure_ascii=False, indent=4)

Если обучить модель на тексте где слова будут заменены на словарную форму или если убрать множество суффиксов, то мы получим более точную и полезную модель.

Можно попробовать смешать подходы и собрать текст с разными модификациями слов:

1. Текст где слова будут без суффиксов.
2. Текст где слова будут без префиксов.
3. Текст где слова будут без суффиксов и префиксов.
4. Текст где слова будут в словарной форме.

То есть, из одного предложения мы можем собрать множество предложений с разными модификациями отделенных слов.

Предположительно, так вес корневой морфемы будет больше и модель будет лучше.

  